## Xây dựng mô hình đề xuất sản phẩm bằng Collaborative Filtering

#### Chuẩn bị dữ liệu tương tác

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('data/events_processed.csv')

C:\Users\ntnh\AppData\Local\Temp\ipykernel_21196\3165677319.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/events_processed.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884964 entries, 0 to 884963
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   event_time            884964 non-null  object 
 1   event_type            884964 non-null  object 
 2   product_id            884964 non-null  int64  
 3   category_id           884964 non-null  int64  
 4   category_code         884964 non-null  object 
 5   brand                 884964 non-null  object 
 6   price                 884964 non-null  float64
 7   user_id               884964 non-null  int64  
 8   user_session          884964 non-null  object 
 9   year                  884964 non-null  int64  
 10  month                 884964 non-null  int64  
 11  day                   884964 non-null  int64  
 12  quarter               884964 non-null  int64  
 13  week                  884964 non-null  int64  
 14  weekday               884964 non-null  int64  
 15  

In [5]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [ ]:
interaction_weights = {
    'view': 1,
    'cart': 3,
    'purchase': 5
}

df['event_strength'] = df['event_type'].map(interaction_weights)

interaction_df = df.groupby(['user_id', 'product_id'])['event_strength'].sum().reset_index()
interaction_df

,user_id,product_id,event_strength
0,1515915625353226922,4101974,1
1,1515915625353230067,3506650,1
2,1515915625353230683,124883,1
3,1515915625353230683,125325,1
4,1515915625353230683,254763,4
...,...,...,...
556632,1515915625611023671,3829355,1
556633,1515915625611023730,953226,1
556634,1515915625611024014,1715907,1
556635,1515915625611024020,4170534,1


#### Xây dựng ma trận thưa user-product

In [23]:
from scipy.sparse import coo_matrix

# Ánh xạ user và product sang chỉ số
user_map = {user_id: idx for idx, user_id in enumerate(interaction_df['user_id'].unique())}
product_map = {product_id: idx for idx, product_id in enumerate(interaction_df['product_id'].unique())}

# Tạo ánh xạ ngược để giải mã sau này
user_map_rev = {idx: user_id for user_id, idx in user_map.items()}
product_map_rev = {idx: product_id for product_id, idx in product_map.items()}

# Map user_id and product_id to matrix indices
interaction_df['user_idx'] = interaction_df['user_id'].map(user_map)
interaction_df['product_idx'] = interaction_df['product_id'].map(product_map)

# Build sparse matrix
matrix = coo_matrix((
    interaction_df['event_strength'], 
    (interaction_df['user_idx'], interaction_df['product_idx'])
))

#### Huấn luyện ALS model

In [31]:
from implicit.als import AlternatingLeastSquares

user_items = matrix.tocsr()
model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(user_items)

  0%|          | 0/20 [00:00<?, ?it/s]

#### Đề xuất 5 sản phẩm cho một người dùng

In [47]:
user_id = 1515915625519388267
user_internal_id = user_map[user_id]
product_idxs, scores = model.recommend(user_internal_id, user_items[user_internal_id], N=5)
decoded_recommended = [product_map_rev[i] for i in product_idxs]
print(f"Top 5 sản phẩm gợi ý cho user {user_id}:", decoded_recommended)

Top 5 sản phẩm gợi ý cho user 1515915625519388267: [1255165, 3537321, 564832, 677289, 886914]
